In [1]:
import pandas as pd
import numpy as np

In [11]:
class ReviewModel():
    #y_after_predict: 3-dimension array
    #y_after_predict[model type index][model series squecence in specific hypa index][y_value_after_predict index]
    y_after_predict = [[[]]]
    val_y = pd.Series([],dtype=pd.StringDtype()) 
    
    bestModel = [[]] # binary, mark index of the best
    #Distance between y_after_predict and val_y
    distance = [[]]
    #Index of <Model type> element in y_after_predict have the best result
    best = 0
    def __init__(self,val_y,y_after_predict):
        self.val_y = val_y
        self.y_after_predict = y_after_predict
#         self.bestModel.reshape(y_after_predict.shape)
#         self.minDistance.reshape(y_after_predict.shape)
        pass
    def HausdoffDistanceBetween_xAndA(self,x,A):
        minn = float('inf')
        for a in A:
            if abs(x-a) < minn:
                minn = abs(x-a)
        return minn
    def HausdoffDistanceBetween_AAndB(self,A,B):
        maxx = -float('inf')
        max1 = -float('inf')
        max2 = -float('inf')
        for a in A:
            temp = self.HausdoffDistanceBetween_xAndA(a,B)
            if temp > max1:
                max1 = temp
        for b in B:
            temp = self.HausdoffDistanceBetween_xAndA(b,A)
            if temp > max2:
                max2 = temp
        if max1 > max2:
            maxx = max1
        else:
            maxx = max2
        return maxx
        pass
    def chooseTheBestHyperParameter(self):
        distance = [0]
        binaryArray=[0]
        self.distance.clear()
        self.bestModel.clear()
        for modelType in self.y_after_predict:
            distance.clear()
            binaryArray.clear()
            for modelHypa in modelType:
                distance.append(self.HausdoffDistanceBetween_AAndB(modelHypa,val_y))
                binaryArray.append(0)
                print(distance)
            self.distance.append(distance.copy())
            binaryArray[distance.index(min(distance))] = 1
            self.bestModel.append(binaryArray.copy())
    
    def chooseTheBestModel(self):
        i = 0
        distance = []
        distance.clear()
        for modelType in self.y_after_predict:
            # print series have the best result in this model type
#             print(modelType[self.bestModel[i].index(max(self.bestModel[i]))])
#             modelType[self.bestModel[i].index(max(self.bestModel[i]))]
            # print min distance in this model type
#             print(self.distance[i][self.bestModel[i].index(max(self.bestModel[i]))])
            distance.append(self.distance[i][self.bestModel[i].index(max(self.bestModel[i]))])
            i += 1
        self.best = distance.index(min(distance))
        pass
    def calculate(self):
        self.chooseTheBestHyperParameter()
        self.chooseTheBestModel()
        return self.bestModel, self.best
            
    def __del__(self):
        pass
    pass

In [12]:
val_y = np.array([10,2,3])
# df_1 = pd.DataFrame()
# df_1[0] = [6,7,8]
# df_1[1] = [2,3,4]
# df_2 = pd.DataFrame()
# df_2[0] = [4,5,3]
# df_2[1] = [9,7,2]
arr = np.array([10, 20, 30, 4, 5, 6, 7, 0, 9, 10, 11, 12, 2, 1, 3, 3, 2, 1])

newarr = arr.reshape(2, 3, 3)
newarr
# df[0].reshape(df_1.shape)
# df[1].reshape(df_2.shape)
# df[0] = df_1
# df[1] = df_2

array([[[10, 20, 30],
        [ 4,  5,  6],
        [ 7,  0,  9]],

       [[10, 11, 12],
        [ 2,  1,  3],
        [ 3,  2,  1]]])

In [13]:
test = ReviewModel(val_y, newarr)
test.val_y = pd.Series([1,2,3])

In [14]:
test.chooseTheBestHyperParameter()

[20]
[20, 4]
[20, 4, 3]
[8]
[8, 7]
[8, 7, 7]


In [15]:
test.distance

[[20, 4, 3], [8, 7, 7]]

In [16]:
test.bestModel

[[0, 0, 1], [0, 1, 0]]

In [17]:
test.chooseTheBestModel()

In [18]:
test.best

0

In [21]:
binaryArray,bestIndex = test.calculate()

[20]
[20, 4]
[20, 4, 3]
[8]
[8, 7]
[8, 7, 7]


In [22]:
binaryArray

[[0, 0, 1], [0, 1, 0]]

In [23]:
bestIndex

0